In [ ]:
import numpy as np
from astropy.io import fits
import astropy.time
import astropy.units as u
from astropy.coordinates import SkyCoord, EarthLocation
import datetime

In [ ]:
"""
;============================================
; FUNCTION GET_HELIO
; Use hfits fx commands to get info out of header and make helio(bary)centric correction



FUNCTION GET_HELIO, hdr, degrees=degrees, barycentric=barycentric, force=force, quiet=quiet

	IF N_ELEMENTS(barycentric) EQ 0 THEN barycentric = 1

	; telescope longitude and latitude 
	tel = FXPAR(hdr,'TELESCOP')
	CASE tel OF
	  'NASA IRTF': BEGIN
		lat = 19. + 49./60. + 34.38594/3600. ; north
		lon = 360. - (155. + 28./60. + 19.19564/3600.) ; west
		alt = 13674.76*0.3048 ; feet to meters
		date = FXPAR(hdr, 'DATE_OBS')
		time = FXPAR(hdr, 'TIME_OBS')
		degrees = 0
		END	
	  'Baade': BEGIN
		; helio already in header
		IF ~KEYWORD_SET(force) THEN RETURN, FXPAR(hdr, 'HELIO')

		print, "GET_HELIO: calculating from header info for FIRE."
		lat = FXPAR(hdr, 'SITELAT')
		lon = FXPAR(hdr, 'SITELONG')
		alt = FXPAR(hdr, 'SITEALT')
		date = FXPAR(hdr, 'DATE-OBS')
		time = FXPAR(hdr, 'UT-TIME')
		degrees = 1				
		END
	   'TILLINGHAST': BEGIN
		;
		IF ~KEYWORD_SET(force) THEN RETURN, FXPAR(hdr, 'BCV')
		message, 'GET_HELIO: Not implemented.'
		END
	   ELSE: message, 'Telescope invalid'
	ENDCASE


	; from date and UT time, get Julian day
	date_ex = STRSPLIT(date,'-',/EXTRACT) ; 0 = year, 1 = month, 2 = day
	time_ex = STRSPLIT(time,':',/EXTRACT) ; 0 = hour, 1 = min, 2 = sec
	jdate = julday(date_ex[1],date_ex[2],date_ex[0],time_ex[0],time_ex[1],time_ex[2])

	; ra and dec in degrees
	ra = FXPAR(hdr, 'RA')
	dec = FXPAR(hdr, 'DEC')
	IF NOT KEYWORD_SET(degrees) THEN BEGIN
		ra_ex = STRSPLIT(ra,'[+:]',/EXTRACT)
		ra_deg = 15 * (ra_ex[0] + ra_ex[1]/60. + ra_ex[2]/3600.)
		dec_ex = STRSPLIT(dec,'[+-:]',/EXTRACT)
		dec_deg = dec[0] + dec_ex[1]/60. + dec_ex[2]/3600.
		; if first character in dec string was negative, make negative
		char = STRMID(dec,0,1)
		IF STRMATCH(char,'\-') EQ 1 THEN dec_deg = -dec_deg
	ENDIF ELSE BEGIN
		ra_deg = ra
		dec_deg = dec
	ENDELSE

	IF KEYWORD_SET(barycentric) THEN BEGIN
		baryvel, jdate, 0, vh, vb
		ra_rad=ra_deg/!RADEG
		dec_rad=dec_deg/!RADEG
		vel = vb[0]*cos(dec_rad)*cos(ra_rad) + $   ;Project velocity toward star
		      vb[1]*cos(dec_rad)*sin(ra_rad) + vb[2]*sin(dec_rad) 
		IF ~KEYWORD_SET(quiet) THEN print, "GET_HELIO: returning barycentric."
		RETURN, vel
	ENDIF ELSE BEGIN
		hcorr = heliocentric(ra_deg,dec_deg,jd=jdate,longitude=lon,latitude=lat, altitude=alt)
		IF ~KEYWORD_SET(quiet) THEN print, "GET_HELIO: returning heliocentric."
		RETURN, -hcorr
	ENDELSE

END

"""

'\n;============================================\n; FUNCTION GET_HELIO\n; Use hfits fx commands to get info out of header and make helio(bary)centric correction\n\n\n\nFUNCTION GET_HELIO, hdr, degrees=degrees, barycentric=barycentric, force=force, quiet=quiet\n\n\tIF N_ELEMENTS(barycentric) EQ 0 THEN barycentric = 1\n\n\t; telescope longitude and latitude \n\ttel = FXPAR(hdr,\'TELESCOP\')\n\tCASE tel OF\n\t  \'NASA IRTF\': BEGIN\n\t\tlat = 19. + 49./60. + 34.38594/3600. ; north\n\t\tlon = 360. - (155. + 28./60. + 19.19564/3600.) ; west\n\t\talt = 13674.76*0.3048 ; feet to meters\n\t\tdate = FXPAR(hdr, \'DATE_OBS\')\n\t\ttime = FXPAR(hdr, \'TIME_OBS\')\n\t\tdegrees = 0\n\t\tEND\t\n\t  \'Baade\': BEGIN\n\t\t; helio already in header\n\t\tIF ~KEYWORD_SET(force) THEN RETURN, FXPAR(hdr, \'HELIO\')\n\n\t\tprint, "GET_HELIO: calculating from header info for FIRE."\n\t\tlat = FXPAR(hdr, \'SITELAT\')\n\t\tlon = FXPAR(hdr, \'SITELONG\')\n\t\talt = FXPAR(hdr, \'SITEALT\')\n\t\tdate = FXPAR(hdr, \

In [ ]:
hdul = fits.open('merged_T452866790.fits')
hdr = hdul[0].header
hdul.close()

 ############################## Xspextool History ############################## [astropy.io.fits.card]
 ############################## Xcombspec History ############################## [astropy.io.fits.card]
 ############################### Xtellcor History ############################## [astropy.io.fits.card]
 ############################# Xmergeorders History ############################ [astropy.io.fits.card]


In [ ]:
def GET_HELIO(hdr, degrees=0, barycentric=0, force=0, quiet=0):

    # step 1: define telescope longitude and latitude
    tel = hdr['TELESCOP']
    if tel == 'NASA IRTF':
        lat = 19. + 49./60. + 34.38594/3600. # north
        lon = 360. - (155. + 28./60. + 19.19564/3600.) # west
        alt = 13674.76*0.3048 # feet to meters
        date = hdr['DATE_OBS']
        time = hdr['TIME_OBS']
        degrees = 0

    # step 2: get time of observation
    date_ex = date.split('-')
    year, month, date = int(date_ex[0]), int(date_ex[1]), int(date_ex[2])
    time_ex = time.split(':')
    hour, minute, second = int(time_ex[0]), int(time_ex[1]), round(float(time_ex[2]))
    dt = datetime.datetime(year, month, date, hour, minute, second)
    obstime = astropy.time.Time(dt)

    # step 3: get ra and dec in degrees if they aren't already
    ra = hdr['TCS_RA']
    dec = hdr['TCS_DEC']
    if degrees == 0:
      ra_ex = ra.split(':')
      ra_deg = 15 * (int(ra_ex[0]) + int(ra_ex[1])/60 + float(ra_ex[2])/3600)
      dec_ex = dec.split(':')
      dec_deg = abs( int(dec_ex[0]) + int(dec_ex[1])/60 + float(dec_ex[2])/3600)
      #if the first character of the dec is negative, then make negative
      if dec[0] == '-':
        dec_deg = -dec_deg
    elif degrees != 0:
      ra_deg = ra
      dec_deg = dec


    # step 4: caculate barycentric or heliocentric correction, depending on keyword
    # see here: https://docs.astropy.org/en/stable/coordinates/velocities.html
    loc = EarthLocation.from_geodetic(lat=lat*u.deg, lon=lon*u.deg, height=alt*u.m)
    sc = SkyCoord(ra=ra_deg*u.deg, dec=dec_deg*u.deg)
    if barycentric == 0:
      heliocorr = sc.radial_velocity_correction('heliocentric', obstime=obstime, location=loc)  
      corr = heliocorr.to(u.km/u.s)  
      if quiet != 0:
        print("GET_HELIO: returning barycentric.")
    else:
      barycorr = sc.radial_velocity_correction('barycentric', obstime=obstime, location=loc)  
      corr = barycorr.to(u.km/u.s)  
      if quiet != 0:
        print("GET_HELIO: returning heliocentric.")

    return corr

In [ ]:
GET_HELIO(hdr)

<Quantity -28.61466074 km / s>